In [ ]:

import random 
import numpy as np # linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_score, completeness_score, \
v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
%matplotlib inline
import os
for dirname, _, filenames in os.walk('../input/sols-data2'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Importation des data

In [ ]:
#Importation des data
data = pd.read_csv('../input/jeuxdedonn/solsdata.csv')
X = data.drop('sols',axis=1).values
y = data['sols'].values

# > > Visualisation des données

In [ ]:
#Pairplot
p=sns.pairplot(data, hue = 'sols')

> ****> Matrice de corrélation

In [ ]:
plt.figure(figsize=(9,6))  # on this line I just set the size of figure to 12 by 10.
p=sns.heatmap(data.corr(), annot=True,cmap ='RdYlGn')  # seaborn has very simple solution for heatmap
plt.title('matrice de corrélation', fontsize = 10)

In [ ]:
data_help = data.copy()

# > Nettoyage des données****

In [ ]:
percentiles = data_help['a'].quantile([0.05,0.95]).values
data_help['a'][data_help['a'] <= percentiles[0]] = percentiles[0]
data_help['a'][data_help['a'] >= percentiles[1]] = percentiles[1]


percentiles = data_help['sci'].quantile([0.05,0.95]).values
data_help['sci'][data_help['sci'] <= percentiles[0]] = percentiles[0]
data_help['sci'][data_help['sci'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['m3'].quantile([0.05,0.95]).values
data_help['m3'][data_help['m3'] <= percentiles[0]] = percentiles[0]
data_help['m3'][data_help['m3'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['m4'].quantile([0.05,0.95]).values
data_help['m4'][data_help['m4'] <= percentiles[0]] = percentiles[0]
data_help['m4'][data_help['m4'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['as'].quantile([0.05,0.95]).values
data_help['as'][data_help['as'] <= percentiles[0]] = percentiles[0]
data_help['ap'][data_help['as'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['sigh'].quantile([0.05,0.95]).values
data_help['sigh'][data_help['sigh'] <= percentiles[0]] = percentiles[0]
data_help['sigh'][data_help['sigh'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['la'].quantile([0.05,0.95]).values
data_help['la'][data_help['la'] <= percentiles[0]] = percentiles[0]
data_help['la'][data_help['la'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['lb'].quantile([0.05,0.95]).values
data_help['lb'][data_help['lb'] <= percentiles[0]] = percentiles[0]
data_help['lb'][data_help['lb'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['H'].quantile([0.05,0.95]).values
data_help['H'][data_help['H'] <= percentiles[0]] = percentiles[0]
data_help['H'][data_help['H'] >= percentiles[1]] = percentiles[1]

percentiles = data_help['alp'].quantile([0.05,0.95]).values
data_help['alp'][data_help['alp'] <= percentiles[0]] = percentiles[0]
data_help['alp'][data_help['alp'] >= percentiles[1]] = percentiles[1]

In [ ]:
X = data_help.drop('sols',axis=1).values
y = data_help['sols'].values

In [ ]:
# On normalise la dataset ainsi conservée
dfnorm = StandardScaler().fit_transform(X)
rows = data_help.shape[0]
cols = data_help.shape[1]
#scaler = StandardScaler()
#Data = scaler.fit_transform(X)

In [ ]:
# On vérifie que la normalisation est un succès
print("Moyennes :\n", np.mean(dfnorm, axis = 0)) # Les moyennes devraient valoir 0 ou être infiniment petites avec l'arrondi
print("\nÉcarts-types :\n", np.std(dfnorm, axis = 0)) # Les écarts-types devraient valoir 1
print("\nMatrice de corrélation :\n", (1/rows) * np.matmul(np.transpose(dfnorm), dfnorm)) # La matrice doit avoir des 1 sur la diagonale

# Analyse en Composantes Principales

In [ ]:
# ACP bénigne afin d'utiliser ses fonctions de calculus de valeurs propres et variances
acp = PCA()
coord=acp.fit_transform(dfnorm)
# Éléments nécessaire au choix du nombre de composantes principales
print("Nombres de composante :\n",acp.n_components_) 
print("Valeurs propres :\n", acp.explained_variance_)
print("\nQualité de représentation des axes en % :\n", acp.explained_variance_ratio_ * 100)
print("\nQualité cumulée en % :\n", np.cumsum(acp.explained_variance_ratio_) * 100)

In [ ]:
# percentage variation 
per_var = np.round(acp.explained_variance_ratio_*100, decimals =1)
labels = ['cp' + str(x) for x in range (1, len(per_var)+1)]
# tracer le pourcentage de variance expliquée par composante principale
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label = labels)
plt.plot(range(1,len(per_var)+1),(acp.explained_variance_ratio_)*100,'r*-')
plt.ylabel('Pourcentage de la variance expliquée')
plt.xlabel('Composante principale')
plt.title('Scree Plot')
plt.show()


In [ ]:
#cumul de variance expliquée critère de kaizer
labels = ['cp' + str(x) for x in range (1, len(per_var)+1)]
plt.plot(labels,np.cumsum(acp.explained_variance_ratio_)*100,'r*-')
plt.title("Variance expliquée vs au nombre de facteurs")
plt.ylabel("Ratio de variance expliquée par le cumul")
plt.xlabel("Numéro du facteur")
plt.show()

In [ ]:
# On génère ici l'ACP sur laquelle nous allons travailler
n_components = 3 # Nombre de composantes principales
acpr = PCA(n_components)
composantes = acpr.fit_transform(dfnorm)

# On affiche les premiers résultats
print("Valeurs propres :\n", acpr.explained_variance_) # On rappelle les valeurs propres conservées
print("\nMatrice de changement de base :\n", acpr.components_) # La matrice de changement de base, obtenue normalement à partir des vecteurs propres
print("\nComposantes principales :\n", composantes) # Les composantes principales données par la matrice de changement de base
print("\nSomme des composantes :\n", composantes.sum(axis = 0)) # La somme par colonne des composantes principales doit valoir 0

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Affichage du dataset ainsi que les 3 premières composante
for i in range(0, n_components):
    data_help['PC' + str(i + 1)] = composantes[:, i]

display(data_help.head())

In [ ]:
#Tableau des trois première composantes princripales
pd.DataFrame({'id':data.sols,'CP1':composantes[:,0], 'CP2':composantes[:,1],'CP3':composantes[:,2]})

# > Choix du nombre de composante

In [ ]:
pca =PCA(3).fit_transform(dfnorm)
pc1=pca[:,0]
pc2=pca[:,1]
pc3=pca[:,2]

# Nuages des individus

In [ ]:
# plot en  2D 'pc1', pc2 (plan1)
def plot_scatter(pc1, pc2):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    species_unique = list(set(y))
    species_colors = ["r","b","g","black","pink","cyan","brown","purple","y","darkblue","orange","purple","turquoise","deeppink"]
    
    for i, spec in enumerate(y):
        plt.scatter(pc1[i], pc2[i], label = spec, s = 20, c=species_colors[species_unique.index(spec)])
        ax.annotate(str(i+1), (pc1[i],pc2[i]))
        from collections import OrderedDict
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), prop={'size': 7}, loc=4)
    
    ax.set_xlabel('Composante principale  1 (50.81)%', fontsize = 15)
    ax.set_ylabel('Composante principale  2 (13.03)%', fontsize = 15)
    ax.axhline(y=0, color="grey", linestyle="--")
    ax.axvline(x=0, color="grey", linestyle="--")
    
    plt.grid()
    plt.axis([-4, 4, -3, 3])
    plt.show()
    
plot_scatter(pc1, pc2)

In [ ]:
# plotting in 2D pc2 pc3 plan(2)
def plot_scatter(pc2, pc3):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    species_unique = list(set(y))
    species_colors = ["r","b","g","black","pink","cyan","brown","purple","y","darkblue","orange","purple","turquoise","deeppink"]

    
    for i, spec in enumerate(y):
        plt.scatter(pc2[i], pc3[i], label = spec, s = 20, c=species_colors[species_unique.index(spec)])
        ax.annotate(str(i+1), (pc2[i],pc3[i]))
        from collections import OrderedDict
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), prop={'size': 7}, loc=4)
    
    ax.set_xlabel('Composante principale  2 (13.03%)', fontsize = 15)
    ax.set_ylabel('Composante principale  3 (11.75)%', fontsize = 15)
    ax.axhline(y=0, color="green", linestyle="--")
    ax.axvline(x=0, color="grey", linestyle="--")
    
    plt.grid()
    plt.axis([-4, 4, -3, 3])
    plt.show()
    
plot_scatter(pc2, pc3)

In [ ]:
# plotting en 2D pc1 & pc3 (plan 3)
def plot_scatter(pc1, pc3):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    species_unique = list(set(y))
    species_colors = ["r","b","g","black","pink","cyan","brown","purple","y","darkblue","orange","purple","turquoise","deeppink"]

    
    for i, spec in enumerate(y):
        plt.scatter(pc1[i], pc3[i], label = spec, s = 20, c=species_colors[species_unique.index(spec)])
        ax.annotate(str(i+1), (pc1[i],pc3[i]))
        from collections import OrderedDict
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), prop={'size': 7}, loc=4)
    
    ax.set_xlabel('Composante principale 1 (50.81)%', fontsize = 15)
    ax.set_ylabel('Composante principale  3 (11.75%)', fontsize = 15)
    ax.axhline(y=0, color="y", linestyle="--")
    ax.axvline(x=0, color="grey", linestyle="--")
    
    plt.grid()
    plt.axis([-4, 4, -3, 3])
    plt.show()
    
plot_scatter(pc1, pc3)

# > Contribution des individus****

> Nous allons  d'abord  calculer les carrés des distances(di^2 à l’origine des individus, qui correspondent également à leur contribution dans l’inertie totale.filtrer les individus avec d_i > à la moyenne de d_i

In [ ]:
# Distance des individus à l'origine
di = np.sum(dfnorm**2,axis=1)
df_ctr_ind = pd.DataFrame({'ID':data.sols,'d_i':di})
df_ctr_ind

In [ ]:
#verification des individus qui ont une bonne distance
df_ctr_ind[df_ctr_ind['d_i'] > df_ctr_ind.d_i.mean()]

Nous allons calculer la contribution de chaque individus dans notre plan 1(CP1 et CP2), plan 2(CP2 et CP3), plan 3(CP1 et CP3). Puis nous déduirons les individus interprétable dans ces plans.

In [ ]:
#contributions aux axes des individus en pourcentage
eigval = (len(data)-1)/len(data)*acp.explained_variance_ # Valeurs propre
col=data_help.shape[1]
ctr = coord**2
for j in range(11):
    ctr[:,j] = ctr[:,j]/((rows)*eigval[j])*100

df_ctr_cp1cp2 = pd.DataFrame({'id':data.sols,'CTR_1':ctr[:,0],'CTR_2':ctr[:,1],'CTR_3':ctr[:,2]})

In [ ]:
#df_ctr_cp1cp2=pd.DataFrame({'id':data.sols,'CTR_1':contrib[:,0],'CTR_2':contrib[:,1],'CTR_3':contrib[:,2]})

In [ ]:
#Tableau de contribution des individus (observations)
df_ctr_cp1cp2=pd.DataFrame({'id':data.sols,'CTR_1':ctr[:,0],'CTR_2':ctr[:,1],'CTR_3':ctr[:,2]})

In [ ]:
#Contribution des individus sur le premier axe 1
df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_1'] > df_ctr_cp1cp2.CTR_1.mean()]

In [ ]:
#Contribution des individus sur le premier axe 2
df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_2'] > df_ctr_cp1cp2.CTR_2.mean()]

In [ ]:
#Contribution des individus sur le premier axe 3
df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_3'] > df_ctr_cp1cp2.CTR_3.mean()]

In [ ]:
#  Contributiondes individu plan 1(axe1 et axe2)
df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_1'] > df_ctr_cp1cp2.CTR_1.mean()].id.append(df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_2'] > df_ctr_cp1cp2.CTR_2.mean()].id).drop_duplicates().reset_index(drop = True)

In [ ]:
# Contribution des individu plan 2(axe2 et axe3)
df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_2'] > df_ctr_cp1cp2.CTR_2.mean()].id.append(df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_3'] > df_ctr_cp1cp2.CTR_3.mean()].id).drop_duplicates().reset_index(drop = True)

In [ ]:
#Contribution des individu plan 3(axe1 et axe3)
df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_1'] > df_ctr_cp1cp2.CTR_1.mean()].id.append(df_ctr_cp1cp2[df_ctr_cp1cp2['CTR_3'] > df_ctr_cp1cp2.CTR_3.mean()].id).drop_duplicates().reset_index(drop = True)

# > Qualité de representation des individus

> Les individus ne sont pas tous bien représentés sur nos composantes principales. Dans un premier temps nous avons afficher tous les individus sur nos axes axes 1, 2 & 3. Nous voulons juste afficher ceux qui seront interprétables. Pour cela nous allons calculer la qualité de représentations d'un individus dans le plan 1 axe1 & axe 2, le plan 2 axe2 & axe 3.
Filtrer les individus ayant une bonne représentation sur nos axes

In [ ]:
# On observe les représentations des individus sur chaque axe
quality = composantes ** 2
for i in range(n_components):
   quality[:,i] = quality[:,i] / np.sum(dfnorm ** 2, axis=1)*100
#print("Pourcentage de représentation des individus par axe :\n", quality * 100)

In [ ]:
#Tableau qualité de représentation des individus
df_ctr_12 = pd.DataFrame({'id':data_help.sols,'COS2_1':quality[:,0],'COS2_2':quality[:,1],'COS2_3':quality[:,2]})

In [ ]:
#Moyenne de chaque COS2
df_ctr_12.mean()

In [ ]:
#Qualité de representation des individus sur l'axe 1
df_ctr_12[df_ctr_12['COS2_1'] > df_ctr_12.COS2_1.mean()]

In [ ]:
#Qualité de representation des individus sur l'axe 2
df_ctr_12[df_ctr_12['COS2_2'] > df_ctr_12.COS2_2.mean()]

In [ ]:
#Qualité de representation des individus sur l'axe 3
df_ctr_12[df_ctr_12['COS2_3'] > df_ctr_12.COS2_3.mean()]

In [ ]:
# Qualité des individu plan 1
df_ctr_12[df_ctr_12['COS2_1'] > df_ctr_12.COS2_1.mean()].id.append(df_ctr_12[df_ctr_12['COS2_2'] > df_ctr_12.COS2_2.mean()].id).drop_duplicates().reset_index(drop = True)

In [ ]:
# Qualité des individu plan 2
df_ctr_12[df_ctr_12['COS2_2'] > df_ctr_12.COS2_2.mean()].id.append(df_ctr_12[df_ctr_12['COS2_3'] > df_ctr_12.COS2_3.mean()].id).drop_duplicates().reset_index(drop = True)

In [ ]:
# Qualité des individu plan 3
df_ctr_12[df_ctr_12['COS2_1'] > df_ctr_12.COS2_1.mean()].id.append(df_ctr_12[df_ctr_12['COS2_3'] > df_ctr_12.COS2_3.mean()].id).drop_duplicates().reset_index(drop = True)

# > Etude des Variables

Analyser le cercle des corrélation et en déduire qu'elles individus seront à droite de l'axe 1 et les individus à gauche
Analyser le cercle des corrélation et en déduire qu'elles individus seront en haut de l'axe 2 et les individus en bas

In [ ]:
#Variables
select=['a','sci','m3','m4','as','ap','sig',
        'la','lb','H','alp']

In [ ]:
#racine carrée des valeurs propres
sqrt_eigval = np.sqrt(acp.explained_variance_)

#corrélation des variables avec les axes
corvar = np.zeros((11,11))
for k in range(11):
    corvar[:,k] = acp.components_[k,:] * sqrt_eigval[k]

In [ ]:
pd.DataFrame(corvar)

In [ ]:
#Corrélation des variable trois première composante
pd.DataFrame({'id':select,'CP1':corvar[:,0],'CP2':corvar[:,1],'CP3':corvar[:,2]})

In [ ]:
#cercle des corrélations plan 1
fig, axes = plt.subplots(figsize=(6,6))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)
#affichage des étiquettes (noms des variables)
for j in range(11):
    plt.annotate(select[j],(corvar[j,0],corvar[j,1]))

#ajouter les axes
plt.plot([-1,1],[0,0],color='cyan',linestyle='--',linewidth=5)
plt.plot([0,0],[-1,1],color='red',linestyle='--',linewidth=5)
axes.set_xlabel('Composante principale 1 (50.81)%', fontsize = 15)
axes.set_ylabel('Composante principale  2 (13.03%)', fontsize = 15)

#ajouter un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False,linewidth=2)
axes.add_artist(cercle)
#affichage
plt.show()

In [ ]:
#cercle des corrélations plan 2
fig, axes = plt.subplots(figsize=(6,6))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)
#affichage des étiquettes (noms des variables)
for j in range(11):
    plt.annotate(select[j],(corvar[j,1],corvar[j,2]))

#ajouter les axes
plt.plot([-1,1],[0,0],color='yellow',linestyle='--',linewidth=5)
plt.plot([0,0],[-1,1],color='green',linestyle='--',linewidth=5)

axes.set_xlabel('Composante principale 2 (13.03%)', fontsize = 15)
axes.set_ylabel('Composante principale  3 (11.75%)', fontsize = 15)

#ajouter un cercle
cercle = plt.Circle((0,0),1,color='red',fill=False,linewidth=2)
axes.add_artist(cercle)
#affichage
plt.show()

In [ ]:
#cercle des corrélations plan 1
fig, axes = plt.subplots(figsize=(6,6))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)
#affichage des étiquettes (noms des variables)
for j in range(11):
    plt.annotate(select[j],(corvar[j,0],corvar[j,2]))

#ajouter les axes
plt.plot([-1,1],[0,0],color='cyan',linestyle='--',linewidth=5)
plt.plot([0,0],[-1,1],color='red',linestyle='--',linewidth=5)
axes.set_xlabel('Composante principale 1 (50.81%)', fontsize = 15)
axes.set_ylabel('Composante principale  3 (10.03%)', fontsize = 15)

#ajouter un cercle
cercle = plt.Circle((0,0),1,color='cyan',fill=False,linewidth=2)
axes.add_artist(cercle)
#affichage
plt.show()

# Qualité de représentation de chaque variable.

In [ ]:
pd.DataFrame(corvar**2)

In [ ]:
#cosinus carré des variables en pourcentages
cos2var = corvar**2
df_ctr_variables = pd.DataFrame({'id':select,'COS2_1':cos2var[:,0],'COS2_2':cos2var[:,1],'COS2_3':cos2var[:,2]})

In [ ]:
#cosinus carré des variables axe 1
df_ctr_variables[df_ctr_variables['COS2_1'] > df_ctr_variables['COS2_1'].mean()]

In [ ]:
#cosinus carré des variables axe 2
df_ctr_variables[df_ctr_variables['COS2_2'] > df_ctr_variables['COS2_2'].mean()]

In [ ]:
#cosinus carré des variables axe 3
df_ctr_variables[df_ctr_variables['COS2_3'] > df_ctr_variables['COS2_3'].mean()]

# > Contribution des variables en %

In [ ]:
cc=(cos2var/acp.explained_variance_)*100

In [ ]:
#Tableau contribution des variables 
df_ctr_variables=pd.DataFrame({"variable":select,"CTR1":cc[:,0],"CTR2":cc[:,1],"CTR3":cc[:,2]})

In [ ]:
#contribution des variables axe1
df_ctr_variables[df_ctr_variables['CTR1'] > df_ctr_variables['CTR1'].mean()]

In [ ]:
#contribution des variables axe2
df_ctr_variables[df_ctr_variables['CTR2'] > df_ctr_variables['CTR2'].mean()]

In [ ]:
#contribution des variables axe3
df_ctr_variables[df_ctr_variables['CTR3'] > df_ctr_variables['CTR3'].mean()]

# Kmeans classification non supervisée

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler
from numpy import median
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score

%matplotlib inline

> Données

In [ ]:
# Données des variables bien represnté de l'ACP
XX=data_help.iloc[:,[0,1,2,3,4,5,7,8,9,10]]
X1=XX.drop('sols',axis=1)
g=XX.describe()
# Données
X_principal = X1.values

In [ ]:
#Choix du nombre de classe
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

model = KMeans()
visualizer = KElbowVisualizer(model, k=(1, 20))

visualizer.fit(X_principal)  
visualizer.show();

In [ ]:
#Application de kmeans au jeu de données / Création du classificateur kmeans
y = data_help['sols'].values
label1= y 
kmeans = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 1000, n_init = 1, random_state = 0)
y_kmeans = kmeans.fit_predict(X_principal)
#index triés des groupes
idk = np.argsort(kmeans.labels_)
#affichage des observations et leurs groupes
pd.DataFrame({"id_sols":data_help.sols[idk],"Cluster":kmeans.labels_[idk]})

In [ ]:
#Visualiser les clusters
fig, axes = plt.subplots(figsize=(6,6))
plt.scatter(X_principal[y_kmeans == 0, 0], X_principal[y_kmeans == 0, 1], s = 100, marker='v', c = 'red', label = 'classe pois')
plt.scatter(X_principal[y_kmeans == 1, 0], X_principal[y_kmeans == 1, 1], s = 100, marker='o', c = 'blue', label = 'Classe labour')
plt.scatter(X_principal[y_kmeans == 2, 0], X_principal[y_kmeans == 2, 1], s = 100, marker='*',c = 'green', label = 'Classe maïs')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s = 150,marker='*', c = 'c', label = 'Centroids')
plt.xlabel('étendue')
plt.ylabel('sci');
plt.legend()

In [ ]:
def k_means(n_clust, data_frame, true_labels):
    """
    La fonction k_means applique l'algorithme de clustering k-means sur le jeu de données et imprime le tableau croisé des étiquettes de cluster et des étiquettes réelles. 
    et les paramètres de performance du clustering.
    
    Entrée :
    n_clust - nombre de clusters (valeur k)
    data_frame - ensemble de données que nous voulons regrouper
    true_labels - étiquettes originales
    
    Sortie :
    1 - tableau croisé des clusters et des étiquettes réelles
    2 - tableau de performance

Traduit avec www.DeepL.com/Translator (version gratuite)
    """
    k_means = KMeans(n_clusters = n_clust,init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    k_means.fit(data_frame)
    c_labels = k_means.labels_
    df = pd.DataFrame({'clust_label': c_labels, 'orig_label': true_labels.tolist()})
    ct = pd.crosstab(df['clust_label'], df['orig_label'])
    
    display(df['clust_label'])
    df['clust_label'].to_csv("submission.csv", index = False)
    y_clust = k_means.predict(data_frame)
    display(ct)
    print('% 9s' % 'inerti  homo    compl   v-meas   ARI     AMI     silhouette')
    print('%i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
      %(k_means.inertia_,
      homogeneity_score(true_labels, y_clust),
      completeness_score(true_labels, y_clust),
      v_measure_score(true_labels, y_clust),
      adjusted_rand_score(true_labels, y_clust),
      adjusted_mutual_info_score(true_labels, y_clust),
      silhouette_score(data_frame, y_clust, metric='euclidean')))

In [ ]:
k_means(3, X_principal, y)

# # Caractérisation des classes (groupes) par les variables

In [ ]:
#moyenne par variable
m = XX.mean()
print("moyenne des variables \n", m )
#TSS
TSS = XX.shape[0]*XX.var(ddof=0)
print(TSS)
#data.frame conditionnellement aux groupes
gb = XX.groupby(kmeans.labels_)
#effectifs conditionnels
nk = gb.size()
print(nk)
#moyennes conditionnelles
mk = gb.mean()
print('Moyennes conditionnelles\n',mk)
#pour chaque groupe écart à la moyenne par variable
EMk = (mk-m)**2
#pondéré par les effectifs du groupe
EM = EMk.multiply(nk,axis=0)
#somme des valeurs => BSS
BSS = np.sum(EM,axis=0)
pd.DataFrame(BSS)
print(BSS)
#carré du rapport de corrélation
#variance expliquée par l'appartenance aux groupes
#pour chaque variable
R2 = BSS/TSS
pd.DataFrame(R2)
print("Le rapport I_in/It \n:", R2)